# 🚍Bus_station
버스 정류장 데이터 전처리

### TODO
- [X] 데이터 불러오기, 필요없는 로우 및 컬럼 일차 삭제  
- [X] 결측값 확인
- [ ] 지도에 시각화
    - [ ] 이상값 확인

### 0. import

In [21]:
import pandas as pd
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt
from IPython.display import set_matplotlib_formats
import koreanize_matplotlib
import folium

### 1. 데이터 불러오기, 필요없는 로우 및 컬럼 일차 삭제  

In [22]:
# 데이터 읽어오기
station = pd.read_csv('bus_station.csv')

station

,정류장번호,정류장명,위도,경도,정보수집일시,모바일단축번호,도시코드,도시명,관리도시명
0,ASB214010001,석근2리 입구,36.935731,127.043094,20221012050123,10001.0,34040,아산시,아산
1,ASB214015410,석근2리 입구,36.935633,127.043204,20221012050123,15410.0,34040,아산시,아산
2,ASB214015456,오성슈퍼,36.951671,127.052784,20221012050123,15456.0,34040,아산시,아산
3,ASB214015459,팽성초교입구,36.952036,127.053031,20221012050123,15459.0,34040,아산시,아산
4,ASB214015461,대사리입구,36.944598,127.053092,20221012050123,15461.0,34040,아산시,아산
...,...,...,...,...,...,...,...,...,...
196004,YSB9505,(가상)남양산TG,35.324320,129.026500,20221012045507,9505.0,38100,양산시,양산
196005,YSB9506,(가상)양산JC,35.329890,129.032600,20221012045507,9506.0,38100,양산시,양산
196006,YSB9507,(가상)구서JC,35.253500,129.100100,20221012045507,9507.0,38100,양산시,양산
196007,YSB9508,(가상)창기마을,35.336660,129.123300,20221012045507,9508.0,38100,양산시,양산


In [23]:
# 역이 도시별로 총 몇 개인지 확인
total_station_cnt_by_city = station["도시명"].value_counts().sort_index()
total_station_cnt_by_city

가평군     966
강릉시      35
강진군       6
거제시    1283
거창군     728
       ... 
해남군    1321
홍성군      11
홍천군     930
화성시    2955
횡성군      24
Name: 도시명, Length: 154, dtype: int64

In [24]:
# 확실히 필요없는 컬럼 제거
station = station.drop(columns=['정보수집일시', '모바일단축번호'])

# 확실히 필요없는 행 제거
# 서울, 인천, 경기 외 지역 제거
station = station[(station['도시명'].isin({'서울특별시', '인천광역시', '수원시', '고양시', '용인시',
                                         '성남시', '안양시', '부천시', '안산시', '화성시', '남양주시', '평택시',
                                         '의정부시', '광명시', '동두천시', '과천시', '구리시', '오산시', '시흥시', '군포시', '의왕시', '하남시', '파주시',
                                         '이천시', '안성시', '김포시', '광주시', '양주시', '포천시', '여주시',
                                         '연천군', '가평군', '양평군'}))]

station = station.reset_index(drop=True)
station

,정류장번호,정류장명,위도,경도,도시코드,도시명,관리도시명
0,GGB100000001,종로2가사거리(중),37.569783,126.987733,11,서울특별시,경기도
1,GGB100000002,창경궁.서울대학교병원(중),37.579233,126.996567,11,서울특별시,경기도
2,GGB100000003,명륜3가.성대입구(중),37.582700,126.998333,11,서울특별시,경기도
3,GGB100000004,종로2가.삼일교(중),37.568683,126.987533,11,서울특별시,경기도
4,GGB100000005,혜화동로터리.여운형활동터(중),37.586233,127.001750,11,서울특별시,경기도
...,...,...,...,...,...,...,...
62534,SEB274121334,청계산(경유),37.440191,127.060532,11,서울특별시,서울
62535,SEB274199480,판교IC(경유),37.399988,127.100572,11,서울특별시,서울
62536,SEB274199481,판교IC(경유),37.401549,127.098467,11,서울특별시,서울
62537,SEB277103813,오도삼거리(경유),37.744490,126.728584,11,서울특별시,서울


In [25]:
# 결측값 존재 확인 
station.info()

# 확인 결과 없음

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 62539 entries, 0 to 62538
Data columns (total 7 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   정류장번호   62539 non-null  object 
 1   정류장명    62539 non-null  object 
 2   위도      62539 non-null  float64
 3   경도      62539 non-null  float64
 4   도시코드    62539 non-null  int64  
 5   도시명     62539 non-null  object 
 6   관리도시명   62539 non-null  object 
dtypes: float64(2), int64(1), object(4)
memory usage: 3.3+ MB
